### Quantum Anomaly Detection for Network Security

We’ll build a binary classifier that detects whether network traffic is normal or anomalous (malicious) using:
- ✅ Classical model: Support Vector Machine (SVM)
- ⚛️ Quantum models:
- QSVC (Quantum Kernel SVM)
- VQC (Variational Quantum Classifier)

All models will be trained on a real-world cybersecurity dataset: [CICIDS2017](https://www.unb.ca/cic/datasets/ids-2017.html).

In [11]:
import pandas as pd

# Load dataset
df = pd.read_csv("CICIDS2017_sample.csv")

# Drop missing values
df = df.dropna()

# Label: convert to binary (0 = Normal, 1 = Attack)
df["Label"] = df["Label"].apply(lambda x: 0 if x == "BENIGN" else 1)

# Separate features and labels
X = df.drop("Label", axis=1).values
y = df["Label"].values

In [12]:
df.count()

Flow Duration             100
Total Fwd Packets         100
Total Backward Packets    100
Flow Bytes/s              100
Flow Packets/s            100
Label                     100
dtype: int64

In [13]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA

# Scale features
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Reduce to 2 dimensions
pca = PCA(n_components=2)
X_reduced = pca.fit_transform(X_scaled)

In [14]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_reduced, y, test_size=0.2, random_state=42)

In [15]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

svc = SVC(kernel="rbf")
svc.fit(X_train, y_train)
y_pred = svc.predict(X_test)

print("SVM Accuracy:", accuracy_score(y_test, y_pred))

SVM Accuracy: 0.55


In [18]:
from qiskit.circuit.library import ZZFeatureMap
from qiskit_machine_learning.kernels import FidelityQuantumKernel
from qiskit_machine_learning.algorithms import QSVC
from qiskit_aer import AerSimulator

# Define 2-qubit feature map with 2 layers (reps=2)
feature_map = ZZFeatureMap(feature_dimension=2, reps=2, entanglement='linear')

# Quantum kernel using Aer simulator
quantum_kernel = FidelityQuantumKernel(feature_map=feature_map, quantum_instance=simulator)

# QSVC model
qsvc = QSVC(quantum_kernel=quantum_kernel)

# Train QSVC
qsvc.fit(X_train, y_train)

# Predict
y_pred_qsvc = qsvc.predict(X_test)

ImportError: cannot import name 'Sampler' from 'qiskit.primitives' (/opt/anaconda3/lib/python3.12/site-packages/qiskit/primitives/__init__.py)